## Intro

<div class="alert alert-success">
This section is up to you and not necessary. 

I would recommend specifing the date you have done the exploration.

in addition you can specify possible problems and doubts regarding both your conclusions and the data itself in this section or in the Coclusions section. (such as biases in data, missing values, small number of samples to support some conclusion, etc)

## Conclusions

<div class="alert alert-success">

**A List of all of your conclusions from the Exploration**
    
No need to specify the actual tests and to explain why you have concluded these things. if the reader wants to deep dive into your tests he will be able to do so further down the notebook.

having said that, **ALWAYS** specify possible problems and doubts regarding both your conclusions and the data itself in this section if they were not mentioned in the Intro! (such as biases in data, missing values, small number of samples to support some conclusion, etc)

</div>

1. ....
2. ....



## Hypothesis & Tests

<div class="alert alert-success">

**This section is where you list out all your assumptions and the way you have tested them.**

It serves both yourself and the reader:
1. The reader - the reader that wants to understand how you have concluded the above conclusions without deep diving into your code.
2. Yourself - It makes it very easy to organize your thoughts and make sure you have tested everything you thought is necessary.

*Keep in mind:*
1. Understand what the data really means in "real life"!!
2. Missing values!
3. Total number of samples
4. Distribution of features
5. Write down all the things you have checked even if they failed - the failure itself may mean allot.
6. Duplicates!

</div>

1. Hypothesis 1: Ticket classes are associated with survival rates. I assume that ticket classes are mapped onto specific locations on the ship, which were directly related to the ship's sinking patterns and thus to chances of surviving. 
    * test 1: explore the distribution of ticket class.
    * test 2: compute percentage of surviving per ticket class. 
    * plot.
 
2. Hypothesis 2: Older age is associated with decreased survival rates.
    * test 1: explore the distrbution of age; should it be re-grouped into a few categories, each covering a different range?
    * test 2: compute age stats (mean, median) under the two survival indices; if I turn age to a categorical variable, then
      compute percentage of surviving per age range. 
    * plot.

3. Hypothesis 3: Passengers who travelled alone are less likely to survive.
    * test 1: explore the distribution of 'sibsp' and 'parch'.
    * test 2: compute percentage of surviving for 'sibsp' and 'parch' (0 vs. >0).
    * plot.

4. Hypothesis 4: Males with families were less likely to survive (i.e. sex by 'sibsp' / 'parch' interaction).
    * test 1: explore the distribution of sex and its combination with 'sibsp' and 'parch'.
    * test 2: compute percentage of surviving for males and females with vs. without 'sibsp' and 'parch' (0 vs. >0).
    * plot.


## Implementation

### Test1

Description of test and the hypothesis

Results and conclusions:
> ...

### Test2

Description of test and the hypothesis

Results and conclusions:
> ...

### Test3

Description of test and the hypothesis

Results and conclusions:
> ...

### Test4

Description of test and the hypothesis

Results and conclusions:
> ...